In [ ]:
import json
import random

In [ ]:
with open("/content/drive/MyDrive/Afstudeerproject/Data/labelled_data.json") as f:
    data = json.load(f)
    random.shuffle(data)

# Levenshtein distance

In [ ]:
!pip install python-Levenshtein
import Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149859 sha256=0856d88183e56022f03cdb144f457cea179e48e2422aa0bfbdce7111290a7c09
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


## Stage 1

In [ ]:
def compare_titles(title1, title2):
    abrv = ""
    titles = [title1, title2]
    for title in titles:
        if len(title.split(" ")) > 1:
            continue

        caps = 0
        for char in title:
            if char.isupper():
                caps += 1

        if caps > 1:
            abrv = title
            titles.remove(abrv)
            title = titles[0]
    
    if (abrv not in title or abrv != "") and (len(abrv) == len(title.split(" ")) or len(title.split(" ")) == 1):
        abrv = abrv.lower()
        char_between = len(title)
        for char in title.lower():
            if len(abrv) == 0:
                title1 = abrv
                title2 = abrv
                break
            elif char == abrv[0] and char_between > 1:
                abrv = abrv[1:]
                char_between = 0
            else:
                char_between += 1
    
    score = Levenshtein.ratio(title1, title2)

    return score

In [130]:
def compare_names(authors1, editors1, authors2, editors2):
    authors1 = set(authors1.split("/"))
    authors2 = set(authors2.split("/"))
    editors1 = set(editors1.split("/"))
    editors2 = set(editors2.split("/"))

    if "" in authors1:
        authors1.remove("")
    if "" in authors2:
        authors2.remove("")
    if "" in editors1:
        editors1.remove("")
    if "" in editors2:
        editors2.remove("")

    print(authors1 | editors1)
    print(authors2 | editors2)
    length1 = len(authors1 | editors1)
    length2 = len(authors2 | editors2)
    total_score = 0

    for author1 in authors1.copy():
        for author2 in authors2:
            name_score = Levenshtein.ratio(author1, author2)
            if name_score > 0.8:
                total_score += name_score
                authors1.remove(author1)
                authors2.remove(author2)
                break
    
    editors1 = editors1 | authors1
    editors2 = editors2 | authors2

    for editor1 in editors1.copy():
        for editor2 in editors2:
            name_score = Levenshtein.ratio(editor1, editor2)
            if name_score > 0.8:
                total_score += name_score
                editors2.remove(editor1)
                editors2.remove(editor2)
                break
    
    return total_score / max([length1, length2])


In [ ]:
def compare_editions(edition1, edition2):
    if type(edition1) == list:
        edition1 = " ".join(edition1)
    if type(edition2) == list:
        edition2 = " ".join(edition2)
    score = Levenshtein.ratio(edition1, edition2)

    return score

In [ ]:
for citation in data:
    if citation["citation"] == "Musielak/Heinrich, ZPO, 7. Aufl. 2009, § 32 Rn. 1":
        provided_citation = citation
        print(citation)
        break

{'citation': 'Musielak/Heinrich, ZPO, 7. Aufl. 2009, § 32 Rn. 1', 'labels': {'title': 'ZPO', 'author': 'Musielak/Heinrich', 'editor': '', 'edition': ['7.', 'Aufl.'], 'year': '2009', 'entry': '§ 32 Rn. 1', 'other': ', , ,'}}


# Complete matching

In [ ]:
def score_citation(citation1, citation2):
    labels1 = citation1["labels"]
    labels2 = citation2["labels"]
    title_score = compare_titles(labels1["title"], labels2["title"])
    name_score = compare_names(labels1["author"], labels1["editor"],
                               labels2["author"], labels2["editor"])
    total_score = (title_score + name_score) / 2
    if title_score > 0.8 or name_score > 0.8:
        edition_score = compare_editions(labels1["edition"], labels2["edition"])
        entry_score = Levenshtein.ratio(labels1["entry"], labels2["entry"])
        total_score += edition_score + entry_score

    return total_score

In [117]:
citation1 = {'citation': 'Musielak/Heinrich, ZPO, 7. Aufl. 2009, § 32 Rn. 1',
   'labels': {'author': 'Heinrich',
    'edition': ['7.', 'Aufl.'],
    'editor': 'Musielak',
    'entry': '§ 32 Rn. 1',
    'other': ', , ,',
    'title': 'Zivilprozessordnung',
    'year': '2009'}}

In [131]:
score_citation(provided_citation, citation1)

{'Heinrich', 'Musielak'}
{'Heinrich', 'Musielak'}


KeyError: ignored

In [ ]:
def match_citation(target_citation):
    with open("/content/drive/MyDrive/Afstudeerproject/Data/labelled_data.json") as f:
        data = json.load(f)
    scored = []
    for citation in data:
        score = score_citation(target_citation, citation)
        scored.append((citation, score))
    
    scored.sort(key=lambda tup: tup[1], reverse=True)

    return scored[:10]

In [ ]:
match_citation(citation1)

[({'citation': 'Musielak/Heinrich, ZPO, 7. Aufl. 2009, § 32 Rn. 1',
   'labels': {'author': 'Musielak/Heinrich',
    'edition': ['7.', 'Aufl.'],
    'editor': '',
    'entry': '§ 32 Rn. 1',
    'other': ', , ,',
    'title': 'ZPO',
    'year': '2009'}},
  3.0),
 ({'citation': 'Musielak/Heinrich, ZPO, 7. Aufl. 2009, § 32 Rn. 7 m.w.N.',
   'labels': {'author': 'Musielak/Heinrich',
    'edition': ['7.', 'Aufl.'],
    'editor': '',
    'entry': '§ 32 Rn. 7',
    'other': ', , , m.w.N.',
    'title': 'ZPO',
    'year': '2009'}},
  2.9),
 ({'citation': 'Musielak-Heinrich, ZPO, 7. Aufl., § 33 Rn. 19',
   'labels': {'author': 'Musielak/Heinrich',
    'edition': ['7.', 'Aufl.'],
    'editor': '',
    'entry': '§ 33 Rn. 19',
    'other': ', , ,',
    'title': 'ZPO',
    'year': ''}},
  2.857142857142857),
 ({'citation': 'Musielak/Heinrich, ZPO 8. Aufl. § 32 Rn. 14',
   'labels': {'author': 'Musielak/Heinrich',
    'edition': ['8.', 'Aufl.'],
    'editor': '',
    'entry': '§ 32 Rn. 14',
    'oth

# To-Do

- Find authors by using __ in editors ✅
- abreviation matching ✅
- Matching authors ✅
- Try clustering
- String matching score: ✅
- Add non-names predictions to matching ✅
- Urteil, beschlu,
- Everything with a known name, no auflage, for validation
-----------------------
- Neglect entry ✅
- Get a list of all the books and titles + possible authors/editors
- Analyze the reference with title, author
- Penatilize missing author ✅
----------------------
- Hierarchical cluster
- Distance as in similarity
- Pre-parse citations
- Random sample and manually label ✅
    - Known names
    - Auflage
    - Save label

- Most important is title, editor

- Staged matching
- Check with ZPO, BGB, StGB

5110 before 1990

552369 court cases
523660 between 1990 - 2021 

2010 everything online and others had to be scraped from different methods


- Writing
    - Abstract 2000
    - Introduction
    - Related work
    - Method
    - In depth explanation
    - Results/Conclusion
    - Discussion
